In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset/IMDB Dataset.xlsx
/kaggle/input/datasetidbm/IMDB Dataset.csv
/kaggle/input/imbd-data/IMDB.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, LSTM 

In [3]:
data = pd.read_csv("/kaggle/input/datasetidbm/IMDB Dataset.csv")

In [4]:
# Exploration du dataset
print(data.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [5]:
# Compter le nombre d'exemples positifs et négatifs dans le dataset
positive_count = data[data['sentiment'] == 'positive'].shape[0]
negative_count = data[data['sentiment'] == 'negative'].shape[0]
print('Nombre d\'exemples positifs :', positive_count)
print('Nombre d\'exemples négatifs :', negative_count)

Nombre d'exemples positifs : 25000
Nombre d'exemples négatifs : 25000


In [6]:
# Afficher un exemple positif
positive_example = data[data['sentiment'] == 'positive']['review'].iloc[0]
print('Exemple positif :', positive_example)


Exemple positif : One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show

In [7]:
# Afficher un exemple négatif
negative_example = data[data['sentiment'] == 'negative']['review'].iloc[0]
print('Exemple négatif :', negative_example)

Exemple négatif : Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.


In [31]:
import re
from string import punctuation

def nettoyer_texte(texte):
    texte = texte.lower()
    texte = ''.join([caractere for caractere in texte if caractere not in punctuation])
    # Supprimer les balises HTML
    texte = re.sub('<.*?>', '', texte)
    return texte

data['review'] = data['review'].apply(nettoyer_texte)

In [32]:
# Tokeniser les textes
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequence_length = 200
padded_sequences = pad_sequences(sequences, maxlen=sequence_length)

In [34]:
data.head()

,review,sentiment,sent_Bi
0,one of the other reviewers has mentioned that ...,positive,1
1,a wonderful little production br br the filmin...,positive,1
2,i thought this was a wonderful way to spend ti...,positive,1
3,basically theres a family where a little boy j...,negative,0
4,petter matteis love in the time of money is a ...,positive,1


4. Construction du Modèle RNN basique

In [37]:
model = Sequential()
embedding_dim = 32
model.add(Embedding(input_dim=10000 , output_dim=embedding_dim, input_length=sequence_length))
lstm_out = 32
model.add(LSTM(lstm_out))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 32)           320000    
                                                                 
 lstm_4 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 328661 (1.25 MB)
Trainable params: 328661 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [39]:
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
313/313 [==============================] - 34s 100ms/step - loss: 0.4020 - accuracy: 0.8073
Epoch 2/10
313/313 [==============================] - 31s 100ms/step - loss: 0.2338 - accuracy: 0.9092
Epoch 3/10
313/313 [==============================] - 31s 100ms/step - loss: 0.1893 - accuracy: 0.9288
Epoch 4/10
313/313 [==============================] - 31s 100ms/step - loss: 0.1555 - accuracy: 0.9425
Epoch 5/10
313/313 [==============================] - 31s 100ms/step - loss: 0.1289 - accuracy: 0.9532
Epoch 6/10
313/313 [==============================] - 32s 101ms/step - loss: 0.1049 - accuracy: 0.9626
Epoch 7/10
313/313 [==============================] - 32s 101ms/step - loss: 0.0902 - accuracy: 0.9684
Epoch 8/10
313/313 [==============================] - 31s 100ms/step - loss: 0.0829 - accuracy: 0.9702
Epoch 9/10
313/313 [==============================] - 31s 100ms/step - loss: 0.0627 - accuracy: 0.9794
Epoch 10/10
313/313 [==============================] - 31s 99ms/step - lo

In [38]:
data['sent_Bi'] = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
y = data['sent_Bi']
X=padded_sequences
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Taille de l\'ensemble d\'entraînement : {len(x_train)}')
print(f'Taille de l\'ensemble de test : {len(x_test)}')

Taille de l'ensemble d'entraînement : 40000
Taille de l'ensemble de test : 10000


In [40]:
# Évaluer la performance du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(x_test, y_test)

# Calculer la précision
precision = accuracy * 100

# Afficher les résultats
print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Precision: ", precision)

313/313 [==============================] - 7s 21ms/step - loss: 0.5207 - accuracy: 0.8675
Loss:  0.5206940174102783
Accuracy:  0.8675000071525574
Precision:  86.75000071525574


The model achieved an accuracy of approximately 86.75%, which means it correctly classified 86.75% of the samples in the test set. The precision is also 86.75%, indicating that the model has a similar proportion of true positive predictions among all the positive predictions it made.

The loss value of 0.5206940174102783 represents the average loss over the test set. A lower loss value indicates that the model's predictions are closer to the true labels.

Overall, these metrics suggest that the model has performed reasonably well on the test set

7. Analyse des Paramètres du Modèle
Expliquer les principaux paramètres du modèle LSTM :
units : Nombre de neurones LSTM dans la couche LSTM.
embedding_dim : Dimension de l'espace d'incorporation (embedding).
input_length : Longueur des séquences en entrée.
activation : Fonction d'activation utilisée dans la dernière couche.

In [50]:
# Créer le modèle RNN basique
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim=10000, output_dim=32, input_length=sequence_length))
model_rnn.add(SimpleRNN(units=))
model_rnn.add(Dense(units=1, activation='sigmoid'))

model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

SyntaxError: invalid syntax (1702512870.py, line 4)

In [ ]:
model_rnn.fit(x_train, y_train, epochs=10, batch_size=128)


Epoch 1/10
281/313 [=========================>....] - ETA: 1s - loss: 0.1547 - accuracy: 0.9405

In [ ]:
# Évaluer la performance du modèle sur l'ensemble de test
loss, accuracy = model_rnn.evaluate(x_test, y_test)

# Calculer la précision
precision = accuracy * 100

# Afficher les résultats
print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Precision: ", precision)